# Assistants API Overview

The new [Assistants API](https://platform.openai.com/docs/assistants/overview) is a stateful evoltuion of our [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) meant to simplify the creation of assistant-like experiences, and enable developer access to powerful tools like Code Interpreter and Knowledge Retreival.

## Assistanst API vs Chat Completions API
TODO...


## Python SDK Setup

> **Note**
> We've updated our Python SDK to add support for the Assistants API functions, so you'll need to update it to the latest version (`1.1.1` at time of writing).

In [ ]:
!pip install --upgrade openai

And make sure it's up to date by running:

In [ ]:
!pip show openai | grep Version

## Complete Example with Assistants API

I'll first define a simple utility function to better visualze the outputs:

In [115]:
import json

def show_json(obj):
    return json.loads(obj.json())

Now we can begin by creating an assistant! We'll create a Math Tutor just like in our [docs](https://platform.openai.com/docs/assistants/overview). We'll specify the following:
- `name`: a high-level name for our Assistant.
- `instructions`: the behavior for our Assistant (similar to the `system` prompt in Chat Completions)
- `tools`: which tools to enable (in this case `code_interpreter`)

In [162]:
from openai import OpenAI


client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_ooPQXw1Jqx0DMH96PEmdvXrw',
 'created_at': 1699473978,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

Next, we'll create a new Thread and add a message to it.

In [117]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_mkOTQm2FbgnFbDwPcj3c25UD',
 'created_at': 1699471265,
 'metadata': {},
 'object': 'thread'}

In [118]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
show_json(message)

{'id': 'msg_PtotfCRWtqqNx8HtdCAGMHVN',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1699471266,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_mkOTQm2FbgnFbDwPcj3c25UD'}

Notice how this Thread is **not** associated with the Assistant we just created! Threads exist independently from Assistants, which may be different from what you'd expect if you've used ChatGPT (where a thread is tied to a model/GPT).

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools. 

> **Note**
> Runs are key difference between the Assistants API and Chat Completions API! While in Chat Completions the model will only ever respond with a single message, in the Assistants API a Run may result in an Assistant using one or multiple tools, and potentially adding multiple messages to the Thread.

To get our Assistant to respond to the user, let's create the Run. As mentioned earlier, you must specify _both_ the Assistant _and_ the Thread.

In [119]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_m4coeyi0QaOxTSvtHBMhD0P6',
 'assistant_id': 'asst_ZMiruz2BFdAUDr5t3Aal0CsU',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699471268,
 'expires_at': 1699471868,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Write and run code to answer math questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_mkOTQm2FbgnFbDwPcj3c25UD',
 'tools': [{'type': 'code_interpreter'}]}

Unlike creating a completion in Chat Completions, creating a Run is an asynchronous operation. It will return immediately with the Run's metadata, which includes a `status` that will initially be set to `queued`. The `status` will be updated as the Assistant performs operations (like using tools and adding messages).

To know when the Assistant has completed processing, we can poll the Run in a loop. (We will soon be adding support for streaming.)

In [120]:
def wait_on_run(run):
    while not (run.status == "completed" or run.status == "failed"):
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
    return run

In [121]:
run = wait_on_run(run)
show_json(run)

{'id': 'run_m4coeyi0QaOxTSvtHBMhD0P6',
 'assistant_id': 'asst_ZMiruz2BFdAUDr5t3Aal0CsU',
 'cancelled_at': None,
 'completed_at': 1699471273,
 'created_at': 1699471268,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Write and run code to answer math questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1699471269,
 'status': 'completed',
 'thread_id': 'thread_mkOTQm2FbgnFbDwPcj3c25UD',
 'tools': [{'type': 'code_interpreter'}]}

Now that the Run has completed we list the Messages in the Thread to see what got added by the Assistant!

In [122]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_hatP0z3rQTyIwV8BPFwBtQmw',
   'assistant_id': 'asst_ZMiruz2BFdAUDr5t3Aal0CsU',
   'content': [{'text': {'annotations': [],
      'value': 'The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).'},
     'type': 'text'}],
   'created_at': 1699471273,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_m4coeyi0QaOxTSvtHBMhD0P6',
   'thread_id': 'thread_mkOTQm2FbgnFbDwPcj3c25UD'},
  {'id': 'msg_PtotfCRWtqqNx8HtdCAGMHVN',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1699471266,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_mkOTQm2FbgnFbDwPcj3c25UD'}],
 'object': 'list',
 'first_id': 'msg_hatP0z3rQTyIwV8BPFwBtQmw',
 'last_id': 'msg_PtotfCRWtqqNx8HtdCAGMHVN',
 'has_more': False}

As you can see, Messages are ordered in reverse-chronological order – this was done so the results are always on the first `page` (since results can be paginated). Do keep a look out for this, since this is opposite to messages in the Chat Completions.

Now let's ask our Assistant to explain.

In [123]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run)

# Get all the messages added after our last user message
messages = client.beta.threads.messages.list(thread_id=thread.id, order='asc', after=message.id)
show_json(messages)

AttributeError: 'list' object has no attribute 'json'

This may feel like a lot of steps to get a response back – however, once you've created your assistant you can re-use it anywhere without having to maintain its state or pass references! It will also show up in your [OpenAI Assistants](https://platform.openai.com/assistants) dashboard.

You can also do all of the setup directly in the dashboard if you prefer!

<img width="1728" alt="Screenshot 2023-11-08 at 3 08 50 PM" src="https://github.com/ibigio/shell-ai/assets/25421602/f8002f51-d3e2-40e4-8ded-f619505ed21e">

Let's take a look at how we could potentially put all of this together! Below is all the code you need to use an Assistant you've created.

In [173]:
from openai import OpenAI

MATH_ASSISTANT_ID = "asst_ooPQXw1Jqx0DMH96PEmdvXrw"

client = OpenAI()

def wait_on_run(thread, run):
    while not (run.status == "completed" or run.status == "failed"):
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
    return run

def submit_message(assistant_id, thread, user_message):
    message = client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )
    wait_on_run(thread, run)
    messages = client.beta.threads.messages.list(
        thread_id=thread.id, order="asc", after=message.id
    )
    return [m.content[0].text.value for m in messages.data]


In [174]:
threads = []
for i in range(3):
    threads.append(client.beta.threads.create())

In [175]:
print(submit_message(MATH_ASSISTANT_ID, threads[0], "I need to solve the equation `3x + 11 = 14`. Can you help me?"))
print(submit_message(MATH_ASSISTANT_ID, threads[1], "Could you explain linear algebra to me?"))
print(submit_message(MATH_ASSISTANT_ID, threads[2], "I don't like math. What can I do?"))


['Subtract 11 from both sides to get `3x = 3`, then divide both sides by 3 to solve for `x`, so `x = 1`.']
['Linear algebra is the branch of mathematics that deals with vectors, vector spaces, linear transformations, and systems of linear equations, often represented using matrices.']
['To make math more enjoyable, try to find aspects of it that relate to your interests, practice regularly to build confidence, or work with a tutor or friend to make the learning process more engaging and supportive.']


# 